In [1]:
import sqlite3
import pandas as pd
import json

pd.options.mode.chained_assignment = None





In [5]:
cnx = sqlite3.connect('./game.db')


In [ ]:
cnx = sqlite3.connect('./game.0802.db')

In [8]:
users = pd.read_sql_query("SELECT * FROM users", cnx)


In [15]:
users.drop_duplicates(keep='last', inplace=True)
users.to_csv('users_db.csv', index=False)

In [11]:
users.loc[users['name']=='Ender']

,id,name,password
22,dd925599ff3e,Ender,202cb962ac59075b964b07152d234b70
5211,dd925599ff3e,Ender,202cb962ac59075b964b07152d234b70


In [75]:

# Python 3 code to demonstrate the 
# working of MD5 (string - hexadecimal)
  
import hashlib
  
# initializing string
str2hash = "ender"
  
# encoding GeeksforGeeks using encode()
# then sending to md5()
result = hashlib.md5(str2hash.encode())
  
# printing the equivalent hexadecimal value.
print("The hexadecimal equivalent of hash is : ", end ="")
print(result.hexdigest())

The hexadecimal equivalent of hash is : 3fa01cb261078d3566dd9dbbbf7e3c63


In [35]:
def get_full_games(cnx):
    df1 = pd.read_sql_query("SELECT * FROM games", cnx)
    df2 = pd.read_sql_query("SELECT * FROM games_202106", cnx)
    df3 = pd.read_sql_query("SELECT * FROM games_202107", cnx)
    df4 = pd.read_sql_query("SELECT * FROM games_202108_202110", cnx)
    df4 = pd.read_sql_query("SELECT * FROM games_202111_202112", cnx)
    print(df1.shape[0], \
        df2.shape[0], \
        df3.shape[0], \
        df4.shape[0], \
        )
    res = pd.concat([df1, df2, df3, df4], axis=0)
    return res

In [54]:
df = get_full_games(cnx).drop_duplicates(keep='first')

12775 974 1765 3266


In [6]:
# df = pd.read_sql_query("SELECT * FROM games", cnx)

In [57]:
df.drop_duplicates(keep='first')

,game_id,save_id,game,status,createtime,prop
0,g4f31beffa9b7,32,"{""exitedPlayers"":[],""activePlayer"":{""id"":""p478...",running,2022-03-10 16:02:43,"{""id"":""g4f31beffa9b7"",""phase"":""action"",""create..."
1,ga2f148f7816e,11,"{""exitedPlayers"":[],""activePlayer"":{""id"":""pa82...",running,2022-03-10 16:02:43,"{""id"":""ga2f148f7816e"",""phase"":""research"",""crea..."
2,ga1b0c90fec3b,0,"{""id"":""ga1b0c90fec3b"",""players"":[{""name"":""2yan...",finished,2022-01-17 14:05:37,None
3,g1e3938413686,0,"{""id"":""g1e3938413686"",""players"":[{""name"":""可爱"",...",finished,2022-01-04 10:19:33,None
4,g1b2f23c93bef,20,"{""exitedPlayers"":[],""activePlayer"":{""id"":""p53a...",running,2022-03-10 16:02:44,"{""id"":""g1b2f23c93bef"",""phase"":""drafting"",""crea..."
...,...,...,...,...,...,...
3261,gb89e8033351f,139,"{""id"":""gb89e8033351f"",""players"":[{""name"":""幼女"",...",finished,2021-12-11 12:58:19,None
3262,g45739795076c,138,"{""id"":""g45739795076c"",""players"":[{""name"":""dwan...",finished,2021-12-26 09:49:58,None
3263,gb8d42f1b45dc,163,"{""id"":""gb8d42f1b45dc"",""players"":[{""name"":""教父"",...",finished,2021-11-29 13:33:59,None
3264,gd9c2cd885b2b,189,"{""id"":""gd9c2cd885b2b"",""players"":[{""name"":""张成"",...",finished,2021-12-24 04:54:40,None


In [110]:
import json

with open('cn.json') as data_file:
    data = json.load(data_file)

json_df = pd.DataFrame.from_dict(data, orient="index").reset_index()
json_df.columns = ['en', 'cn']
json_df.to_csv('./中文翻译.csv', index=False)

In [39]:
game_df = pd.read_sql_query("SELECT * FROM game_results", cnx)
game_df.shape[0]

14372

In [18]:
# 处理game_df

game_df["scores"] = game_df["scores"].astype(object)
game_df["game_options"] = game_df["game_options"].astype(object)
game_df["createtime"] = pd.to_datetime(game_df["createtime"], dayfirst=True)
store = []
for i in game_df["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

options = []
for i in game_df["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

df3 = pd.concat([game_df, df1, df2],axis=1)
df4= df3.drop(columns=['game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers'])
df4.to_csv('preprocess.csv', index=False)


In [58]:
df2

,game_id,seed_game_id,players,generations,game_options,scores,createtime,save_id,game,status,createtime_drop,prop,_merge
6568,g7f967b5a1c33,None,4,8,"{""boardName"":""tharsis"",""undoOption"":true,""show...","[{""corporation"":""Splice (breakthrough)"",""corpo...",2021-06-15 13:03:52,0.0,"{'id': 'g7f967b5a1c33', 'players': [{'name': '...",finished,2021-06-15 09:22:23,NaN,both
6569,g7f967b5a1c33,None,4,8,"{""boardName"":""tharsis"",""undoOption"":true,""show...","[{""corporation"":""Splice (breakthrough)"",""corpo...",2021-06-15 13:03:52,175.0,"{'id': 'g7f967b5a1c33', 'players': [{'name': '...",finished,2021-06-15 13:03:51,NaN,both
6572,ga236647f0eb3,None,4,6,"{""boardName"":""elysium"",""undoOption"":true,""show...","[{""corporation"":""Id Front"",""corporation2"":""Int...",2021-06-15 13:42:15,0.0,"{'id': 'ga236647f0eb3', 'players': [{'name': '...",finished,2021-06-15 10:46:37,NaN,both
6573,ga236647f0eb3,None,4,6,"{""boardName"":""elysium"",""undoOption"":true,""show...","[{""corporation"":""Id Front"",""corporation2"":""Int...",2021-06-15 13:42:15,167.0,"{'id': 'ga236647f0eb3', 'players': [{'name': '...",finished,2021-06-15 13:42:15,NaN,both
6582,g6730beb64a5c,None,4,7,"{""boardName"":""hellas"",""undoOption"":true,""showT...","[{""corporation"":""Pharmacy Union"",""corporation2...",2021-06-15 15:34:02,0.0,"{'id': 'g6730beb64a5c', 'players': [{'name': '...",finished,2021-06-15 13:07:47,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19949,ga168cc919836,None,4,7,"{""boardName"":""random"",""undoOption"":true,""showT...","[{""corporation"":""Pobolog (breakthrough)"",""corp...",2022-03-10 15:52:23,169.0,"{'exitedPlayers': [], 'activePlayer': {'id': '...",finished,2022-03-10 15:52:23,"{""id"":""ga168cc919836"",""phase"":""end"",""createtim...",both
19956,g55c6366d7f47,None,4,5,"{""boardName"":""arabia terra"",""undoOption"":true,...","[{""corporation"":""Lakefront Resorts"",""corporati...",2022-03-10 16:46:54,0.0,"{'exitedPlayers': [], 'activePlayer': {'id': '...",finished,2022-03-10 14:30:25,"{""id"":""g55c6366d7f47"",""phase"":""initial_draftin...",both
19957,g55c6366d7f47,None,4,5,"{""boardName"":""arabia terra"",""undoOption"":true,...","[{""corporation"":""Lakefront Resorts"",""corporati...",2022-03-10 16:46:54,129.0,"{'exitedPlayers': [], 'activePlayer': {'id': '...",finished,2022-03-10 16:46:54,"{""id"":""g55c6366d7f47"",""phase"":""end"",""createtim...",both
19958,g918fcce6146c,None,4,8,"{""boardName"":""tharsis"",""undoOption"":true,""show...","[{""corporation"":""Valley Trust"",""corporation2"":...",2022-03-10 17:07:32,0.0,"{'exitedPlayers': [], 'activePlayer': {'id': '...",finished,2022-03-10 14:53:43,"{""id"":""g918fcce6146c"",""phase"":""research"",""crea...",both


In [40]:
# 选择所有已完成数据
df1a = df[df['status']=='finished']
print(df1a.shape[0])

#  关联快照表，取所需字段，即game_df可以通过streamlit选项动态调整
df1b = game_df.merge(df1a,how='left',on=['game_id', 'game_id'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

df2 = df1b[df1b['players'] == 4]
# 修改需要展开字段的类型为list
false = False
true = True
null = None
df2['game'] = df2['game'].apply(lambda x:eval(x))
mid = pd.json_normalize(df2['game'],record_path=['players', 'playedCards'], meta=['id',  ['players','terraformRating'], ['players','name']], sep='_')




11498


In [26]:
mid

,name,resourceCount,bonusResource,targetCards,plantCardsNames,corporationCardsNames,cost,tags,cardType,moonCardProperties_reserveUnits_megacredits,...,buyingTerms_to,buyingTerms_limit,sellingTerms_from,sellingTerms_to,sellingTerms_limit,tilesBuilt,cloneTag,id,players_terraformRating,players_name
0,Early Settlement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g7f967b5a1c33,31,点点
1,Nitrogen Shipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g7f967b5a1c33,31,点点
2,Research Outpost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g7f967b5a1c33,31,点点
3,Great Escarpment Consortium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g7f967b5a1c33,31,点点
4,Titanium Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g7f967b5a1c33,31,点点
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190181,Cloud Seeding,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g918fcce6146c,30,啸
190182,Grass,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g918fcce6146c,30,啸
190183,Ice Cap Melting,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g918fcce6146c,30,啸
190184,Worms,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g918fcce6146c,30,啸


In [59]:
mid1 = mid.loc[:,['id','players_name','name']]
mid1['rn'] = mid1 \
            .groupby(['id','players_name']) \
            .cumcount() + 1
# mid
# TODO  需要关联处理后的game_result表，获取position, gens, createtime等数据, 只能通过merge实现

In [98]:
def getPlayerNumResult(df, player_num = 4):
    """
    主键: game_id, player
    """
    df = df.loc[df['players'] == player_num].reset_index(drop=True)
    for i in range(1, player_num+1):
        player_idx = 'player'+str(i)
        # player_df_pre = df[player_idx].apply(lambda x:eval(x))
        # print(player_idx)
        # player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
        player_df = pd.json_normalize(df[player_idx]).reset_index(drop=True)
        if i == 1:
            res = pd.concat([df,player_df.reindex(df.index)],axis=1)
            print((res.loc[pd.isna(res['player']) == False]).shape[0])
        else:
            mid = pd.concat([df,player_df.reindex(df.index)],axis=1)
            res = pd.concat([res, mid],axis=0, ignore_index=True)
            # print((mid.loc[pd.isna(mid['player']) == False]).shape[0])
        # df = pd.concat([df, pd.json_normalize(df[player_idx])],axis=1)
    res.drop(['player'+str(i) for i in range(1, 7)], axis=1, inplace=True)
    res['count'] = 1
    return res

def getCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby('name') \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left').drop('en', axis = 1)
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = '暂无翻译'
    return res

def getPlayersCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby(['player', 'name']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum'),
        sum_position = ('position', 'sum'),
        sum_playerScore = ('playerScore', 'sum'),
        sum_generations = ('generations', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left').drop('en', axis = 1)
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = '暂无翻译'
    return res

In [42]:
player_df = getPlayerNumResult(df4) # 主键: game_id, player

card_df = mid1.merge(player_df, how='left', left_on=['id', 'players_name'], right_on=['game_id', 'player'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')


4431


In [69]:
card_df[card_df['player'] == '钝钝']

,id,players_name,name,rn,game_id,seed_game_id,players,generations,createtime,preludeExtension,...,escapeVelocityPenalty,corporation,playerScore,player,order,position,userId,corporation2,count,_merge
117,ga236647f0eb3,钝钝,UNMI Contractor,1,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
118,ga236647f0eb3,钝钝,Metal-Rich Asteroid,2,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
119,ga236647f0eb3,钝钝,Soletta,3,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
120,ga236647f0eb3,钝钝,Olympus Conference,4,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
121,ga236647f0eb3,钝钝,Optimal Aerobraking,5,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168154,gc2ac3fd5df38,钝钝,Herbivores,35,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168155,gc2ac3fd5df38,钝钝,Heather,36,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168156,gc2ac3fd5df38,钝钝,Advanced Ecosystems,37,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168157,gc2ac3fd5df38,钝钝,Trans-Neptune Probe,38,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both


In [43]:
def getPlayerCardRank(df, player):
    ff1 = df.query('player == "%s"'%(player))
    return getCardRank(ff1)

In [111]:
card_df_group = getCardRank(card_df)
getPlayersCardRank(card_df).to_csv('playersCardRank.csv', index=False)
card_df_group.sort_values('total',ascending=False).to_csv('./allCardsRank.csv', index = False)

In [74]:
card_rank = getPlayersCardRank(card_df)
card_rank[card_rank['player'] == 'ender']

,player,name,position,playerScore,generations,total,sum_position,sum_playerScore,sum_generations
2707,ender,Great Dam:promo,1.0,86.0,6.0,1,1,86,6
2710,ender,Floater-Urbanism,1.0,86.0,6.0,1,1,86,6
2713,ender,Diaspora Movement,1.0,86.0,6.0,1,1,86,6
2717,ender,Birds,1.0,86.0,6.0,1,1,86,6
2718,ender,Atmoscoop,1.0,86.0,6.0,1,1,86,6
...,...,...,...,...,...,...,...,...,...
89593,ender,Heather,4.0,85.0,6.0,1,4,85,6
89594,ender,Geological Expedition,4.0,85.0,6.0,1,4,85,6
89595,ender,GMO Contract,4.0,85.0,6.0,1,4,85,6
89596,ender,Floater Leasing,4.0,85.0,6.0,1,4,85,6


In [80]:
card_df_group.sort_values('total',ascending=False).to_csv('./allCardsRank.csv', index = False)


In [97]:
false = False
true = True
player_df_new = df2['game'].apply(lambda x:eval(x))

In [143]:
player_df_new

6950    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7020    {'id': 'g32498849988e', 'players': [{'name': '...
7068    {'id': 'gd4c2a4f2bbf3', 'players': [{'name': '...
7083    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7089    {'id': 'g3855ccb42bec', 'players': [{'name': '...
Name: game, dtype: object

In [156]:
# method1
mid = pd.json_normalize(player_df_new,record_path=['players', 'playedCards'], meta=[['players','terraformRating'], ['players','name']], sep='_')
mid
# pd.json_normalize(mid.iloc[0],record_path=['playedCards'])


,name,resourceCount,bonusResource,players_terraformRating,players_name
0,Biolab,NaN,NaN,33,米饭
1,Aerospace Mission,NaN,NaN,33,米饭
2,Nitrate Reducers,NaN,NaN,33,米饭
3,Psychrophiles,1.0,NaN,33,米饭
4,Invention Contest,NaN,NaN,33,米饭
...,...,...,...,...,...
115,Underground City,NaN,NaN,56,You
116,Algae,NaN,NaN,56,You
117,Sub-Crust Measurements,NaN,NaN,56,You
118,Galilean Waystation,NaN,NaN,56,You


In [120]:
# pd.json_normalize(player_df_new,record_path=['players'],meta=['id'])

# mid = pd.json_normalize(player_df_new,record_path=['players'])
mid = pd.json_normalize(player_df_new)['players']
# pd.json_normalize(mid['players'],record_path=['playedCards'],meta=['name',['_game','id']], record_prefix='*')
# pd.json_normalize(mid['players'],record_path=['playedCards'])

In [133]:
mid

0    [{'name': 'You', 'color': 'red', 'beginner': F...
1    [{'name': '米饭', 'color': 'red', 'beginner': Fa...
2    [{'name': 'You', 'color': 'red', 'beginner': F...
3    [{'name': 'You', 'color': 'red', 'beginner': F...
4    [{'name': 'You', 'color': 'red', 'beginner': F...
Name: players, dtype: object

In [134]:
mid = mid.tail(1)

In [139]:
mid.iloc[0]

[{'name': 'You',
  'color': 'red',
  'beginner': False,
  'handicap': 0,
  '_game': {'id': 'g3855ccb42bec'},
  'corpCard': {'name': 'Saturn Systems'},
  'pickedCorporationCard': {'name': 'Saturn Systems'},
  'terraformRating': 56,
  'hasIncreasedTerraformRatingThisGeneration': True,
  'terraformRatingAtGenerationStart': 14,
  'megaCredits': 63,
  'megaCreditProduction': 7,
  'steel': 4,
  'steelProduction': 4,
  'titanium': 1,
  'titaniumProduction': 1,
  'plants': 3,
  'plantProduction': 11,
  'energy': 3,
  'energyProduction': 3,
  'heat': 15,
  'heatProduction': 13,
  'titaniumValue': 3,
  'steelValue': 2,
  'canUseHeatAsMegaCredits': False,
  'actionsTakenThisRound': 0,
  'actionsThisGeneration': [],
  'undoing': False,
  'exited': False,
  'canExit': False,
  'corpInitialActionDone': False,
  'corp2InitialActionDone': False,
  'dealtCorporationCards': [],
  'dealtProjectCards': [],
  'dealtPreludeCards': [],
  'cardsInHand': [{'resourceCount': 0, 'name': 'Martian Zoo'},
   {'resou

In [140]:
pd.json_normalize(mid.iloc[0])

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,heatProductionStepsIncreasedThisGeneration,id,_game.id,corpCard.name,pickedCorporationCard.name,timer.sumElapsed,timer.startedAt,timer.running,timer.afterFirstAction,timer.lastStoppedAt
0,You,red,False,0,56,True,14,63,7,4,...,0,p78988c7af87e,g3855ccb42bec,Saturn Systems,Saturn Systems,1430484,1627868526517,False,True,1627868527001


In [141]:
pd.json_normalize(mid.iloc[0],record_path=['playedCards'])

,name,resourceCount,bonusResource
0,Mohole Excavation,NaN,NaN
1,Business Empire,NaN,NaN
2,Regolith Eaters,0.0,NaN
3,Convoy From Europa,NaN,NaN
4,Subterranean Reservoir,NaN,NaN
5,Lichen,NaN,NaN
6,Mohole Area,NaN,NaN
7,Trade Envoys,NaN,NaN
8,Deep Well Heating,NaN,NaN
9,Peroxide Power,NaN,NaN


In [64]:
false = False
true = True
player_df_pre = df2['game'].apply(lambda x:eval(x))
# print(player_idx)
player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
player_df = player_df.loc[:,['id', 'players']]
# pd.DataFrame([flatten_json(x) for x in eval(df.iloc[0,2])])

In [67]:
# player_df['players'] = player_df['players'].apply(lambda x:eval(x))
player_df2 = player_df.explode('players')
# player_df3a = player_df2['players'].apply(lambda x:eval(x))
# print(player_idx)
player_df3 = pd.json_normalize(player_df2).reset_index(drop=True)

In [91]:
# player_df2['players'] = eval(player_df2['players'])
player_df2['players']


0    {'name': 'You', 'color': 'red', 'beginner': Fa...
1    {'name': '米饭', 'color': 'red', 'beginner': Fal...
1    {'name': '小麦', 'color': 'green', 'beginner': F...
2    {'name': 'You', 'color': 'red', 'beginner': Fa...
3    {'name': 'You', 'color': 'red', 'beginner': Fa...
4    {'name': 'You', 'color': 'red', 'beginner': Fa...
Name: players, dtype: object

In [89]:
player_df2['players'] = player_df2['players'].astype(str).apply(lambda x:eval(x))

In [122]:
f = open('./count.txt')
count = f.read()
f.close()
f = open('./count.txt', 'w')
add = str(int(count)+1)
f.write(add)
f.close()


In [93]:
player_df3 = pd.json_normalize(player_df2['players'])[:,'']

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,waitingFor.options,userId,corpCard.name,pickedCorporationCard.name,corpCard.tags,corpCard.startingMegaCredits,corpCard.cardType,pickedCorporationCard.tags,pickedCorporationCard.startingMegaCredits,pickedCorporationCard.cardType
0,You,red,False,0,14,False,14,0,0,0,...,"[{'title': 'Select corporation', 'buttonLabel'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,米饭,red,False,0,33,True,20,58,21,1,...,NaN,d484c04d1ffb,Valley Trust (breakthrough),Valley Trust (breakthrough),NaN,NaN,NaN,NaN,NaN,NaN
2,小麦,green,False,0,52,True,20,71,15,0,...,NaN,dfc7613bd132,Interplanetary Cinematics (breakthrough),Interplanetary Cinematics (breakthrough),[building],50.0,corporation,[building],50.0,corporation
3,You,red,False,0,62,True,14,82,7,12,...,NaN,NaN,EcoLine,EcoLine,NaN,NaN,NaN,NaN,NaN,NaN
4,You,red,False,0,37,True,14,56,11,12,...,NaN,NaN,CrediCor,CrediCor,NaN,NaN,NaN,NaN,NaN,NaN
5,You,red,False,0,56,True,14,63,7,4,...,NaN,NaN,Saturn Systems,Saturn Systems,NaN,NaN,NaN,NaN,NaN,NaN
